<a href="https://colab.research.google.com/github/pgadosey/pytorch-mobilenet/blob/master/SpeedingUpCNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Speeding up Convolutional Neural Networks

>>>![](https://cdn-images-1.medium.com/max/716/1*FjzcTRoe-R680V0hOwYo5A.png)

>>>>> From “Speeding-up Convolutional Neural Networks Using Fine-tuned CP-Decomposition” paper

This notebook is meant to support [this](https://medium.com/@alexburlacu1996/speeding-up-neural-networks-convolutions-240beac5e30f) blog post with concrete examples.

In [0]:
import tensorflow as tf
import keras as _k

print(f"Keras version: {_k.__version__}")
print(f"Tensorflow version: {tf.__version__}")

Keras version: 2.1.6
Tensorflow version: 1.7.0


Using TensorFlow backend.


## Models

The notebook explains how different methods of speeding up the convolutional layers work and their effect on a LeNet inspired model and an All-CNN-C model.

In [0]:
import keras.backend as K
from keras.layers import Conv2D, Activation, Lambda, Conv3D, BatchNormalization


ExpandDimension = lambda axis: Lambda(lambda x: K.expand_dims(x, axis))
SqueezeDimension = lambda axis: Lambda(lambda x: K.squeeze(x, axis))

# Layers
def simple_factorized_conv(filters, kernel, *args, **kwargs):
    kwargs["activation"] = None
    def __inner(inp):
        cnn1 = Conv2D(filters, (kernel[0], 1), *args, **kwargs)(inp)
        cnn2 = Conv2D(filters, (1, kernel[1]), *args, **kwargs)(cnn1)
        
        return cnn2

    return __inner
  
def cp_decomposed_conv(filters, kernel, *args, **kwargs):
    """
    Beware, it doesn't work! It's just an aproximate demostration of how
    CP decomposition should be implemented. The issue is in the dimension matching.
    If you found the solution, feel free to comment it, either in the blogpost or here.
    You'll get full credit for your finding.
    """
    kwargs["activation"] = None
    rank = filters // 2
    d = kernel[0]
    def __inner(inp):
        first    = Conv2D(rank, kernel_size=(1, 1), **kwargs)(inp)
        
        expanded = ExpandDimension(axis=1)(first)
        mid1     = Conv3D(rank, kernel_size=(d, 1, 1), **kwargs)(expanded)
        mid2     = Conv3D(rank, kernel_size=(1, d, 1), **kwargs)(mid1)
        squeezed = SqueezeDimension(axis=1)(mid2)
        
        last     = Conv2D(filters,  kernel_size=(1, 1), **kwargs)(squeezed)
        
        return last

    return __inner

def bn_relu(layer):
    def __inner(*args, **kwargs):
        l = layer(*args, **kwargs)
        bn = BatchNormalization()(l)
        act = Activation("relu")(bn)
        
        return bn

    return __inner


In [0]:
from keras.regularizers import l2
from keras.models import Model
from keras.layers import (
    Dense, Dropout, Conv2D, SeparableConv2D,
    MaxPool2D, Flatten, GlobalAveragePooling2D
)

class BaseExampleModel:
    def __init__(self, conv_type="std"):
        """
        :convtype: used as a flag to easily switch the Convolution layer implementation
        """
        self.conv_layer = lambda *args, **kwargs: {
                              "std": bn_relu(Conv2D(*args, **kwargs)),
                              "sep": bn_relu(SeparableConv2D(*args, **kwargs)),
                              "cp" : bn_relu(cp_decomposed_conv(*args, **kwargs)),
                              "fac": bn_relu(simple_factorized_conv(*args, **kwargs))
                          }[conv_type]
        
    def make(self, inp, nb_classes):
        """
        :inp: a reference to the Keras Input layer, to easily switch the input size
              and even the way data is fed into network
              (via standard Keras methods or via TF Dataset API)
        :nb_classes: for the final Dense layer
        """
        NotImplemented

class AllCNNLike(BaseExampleModel):
    """
    isnpired from https://arxiv.org/abs/1412.6806
    namely All-CNN-C
    """
    def __init__(self, conv_type="std"):
        super().__init__(conv_type=conv_type)
        
    def make(self, inp, nb_classes):
        conv1 = self.conv_layer(96, (3, 3), padding="same")(inp)
        conv2 = self.conv_layer(96, (3, 3), padding="same")(conv1)
        
        conv3 = bn_relu(Conv2D(96, (3, 3), padding="same", strides=(2, 2)))(conv2)
        
        conv4 = self.conv_layer(192, (3, 3), padding="same")(conv3)
        conv5 = self.conv_layer(192, (3, 3), padding="same")(conv4)
        
        conv6 = bn_relu(Conv2D(192, (3, 3), padding="same", strides=(2, 2)))(conv5)
        
        conv7 = bn_relu(Conv2D(192, (3, 3), padding="same"))(conv6)
        conv8 = bn_relu(Conv2D(192, (1, 1), padding="same"))(conv7)
        conv9 = bn_relu(Conv2D(nb_classes, (1, 1), padding="same"))(conv8)
        
        gap = GlobalAveragePooling2D()(conv9)
        final = Activation("softmax")(gap)
        
        return Model(inputs=inp, outputs=final)
      
class WiderAllCNNLike(BaseExampleModel):
    """
    isnpired from https://arxiv.org/abs/1412.6806
    namely All-CNN-C
    """
    def __init__(self, conv_type="std"):
        super().__init__(conv_type=conv_type)
        
    def make(self, inp, nb_classes):
        conv1 = self.conv_layer(192, (3, 3), padding="same")(inp)
        
        conv2 = bn_relu(Conv2D(96, (3, 3), padding="same", strides=(2, 2)))(conv1)
        
        conv3 = self.conv_layer(384, (3, 3), padding="same")(conv2)
        
        conv4 = bn_relu(Conv2D(128, (3, 3), padding="same", strides=(2, 2)))(conv3)
        
        conv5 = bn_relu(Conv2D(256, (3, 3), padding="same"))(conv4)
        conv6 = bn_relu(Conv2D(nb_classes, (1, 1), padding="same"))(conv5)
        
        gap = GlobalAveragePooling2D()(conv6)
        final = Activation("softmax")(gap)
        
        return Model(inputs=inp, outputs=final)

In [0]:
from keras.datasets import cifar100
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

BATCH_SIZE = 32

def preprocess_data(pair):
    x, y = pair
    return x / 255., to_categorical(y)

(x_train, y_train), (x_test, y_test) = map(preprocess_data, cifar100.load_data())

train_gen = ImageDataGenerator().flow(x_train, y_train, batch_size=BATCH_SIZE)
test_gen = ImageDataGenerator().flow(x_test, y_test, batch_size=BATCH_SIZE)

In [0]:
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy
from keras.optimizers import  SGD
from keras.layers import Input

def main(conv_type="std", verbose=False, net_type="simple"):
    inp = Input(shape=(32, 32, 3))
    net = {
        "simple": AllCNNLike(conv_type=conv_type).make(inp, nb_classes=100),
        "wide": WiderAllCNNLike(conv_type=conv_type).make(inp, nb_classes=100)
    }[net_type]

    net.compile(SGD(lr=0.01, decay=0.001, momentum=0.8), "categorical_crossentropy",
                 metrics=[categorical_accuracy, top_k_categorical_accuracy])

    if verbose:
        print(net.summary())

    net.fit_generator(train_gen, steps_per_epoch=50000 // BATCH_SIZE, epochs=5, verbose=verbose)
    loss, acc, topk_acc = net.evaluate_generator(test_gen, steps=10000 // BATCH_SIZE)

    print(f"Accuracy: {acc}, Top5 Accuracy {topk_acc} and Loss {loss}.")

## Standard Convolutions

In [0]:
# Run it!
main("std", verbose=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 96)        2688      
_________________________________________________________________
batch_normalization_12 (Batc (None, 32, 32, 96)        384       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 32, 32, 96)        83040     
_________________________________________________________________
batch_normalization_13 (Batc (None, 32, 32, 96)        384       
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 16, 16, 96)        83040     
_________________________________________________________________
batch_normalization_14 (Batc (None, 16, 16, 96)        384       
__________

 756/1562 [=============>................] - ETA: 48s - loss: 4.2137 - categorical_accuracy: 0.0740 - top_k_categorical_accuracy: 0.2384

1491/1562 [===========================>..] - ETA: 4s - loss: 4.1584 - categorical_accuracy: 0.0817 - top_k_categorical_accuracy: 0.2596

1562/1562 [==============================] - 91s 58ms/step - loss: 4.1547 - categorical_accuracy: 0.0822 - top_k_categorical_accuracy: 0.2606
Epoch 2/5
 312/1562 [====>.........................] - ETA: 1:10 - loss: 4.0689 - categorical_accuracy: 0.1015 - top_k_categorical_accuracy: 0.2948

1044/1562 [===================>..........] - ETA: 29s - loss: 4.0462 - categorical_accuracy: 0.1035 - top_k_categorical_accuracy: 0.3005

...

 732/1562 [=============>................] - ETA: 47s - loss: 3.9670 - categorical_accuracy: 0.1200 - top_k_categorical_accuracy: 0.3256

1467/1562 [===========================>..] - ETA: 5s - loss: 3.9596 - categorical_accuracy: 0.1204 - top_k_categorical_accuracy: 0.3268

1562/1562 [==============================] - 90s 57ms/step - loss: 3.9590 - categorical_accuracy: 0.1203 - top_k_categorical_accuracy: 0.3267
Epoch 5/5
 300/1562 [====>.........................] - ETA: 1:11 - loss: 3.9577 - categorical_accuracy: 0.1170 - top_k_categorical_accuracy: 0.3214

1032/1562 [==================>...........] - ETA: 30s - loss: 3.9419 - categorical_accuracy: 0.1221 - top_k_categorical_accuracy: 0.3330

1562/1562 [==============================] - 89s 57ms/step - loss: 3.9379 - categorical_accuracy: 0.1231 - top_k_categorical_accuracy: 0.3336
Accuracy: 0.13661858974358973, Top5 Accuracy 0.34705528846153844 and Loss 3.91607553148881.


## Simply Factorized Convolutions

In [0]:
# You know what to do ;)
main("fac", verbose=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 32, 32, 96)        960       
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 32, 32, 96)        27744     
_________________________________________________________________
batch_normalization_27 (Batc (None, 32, 32, 96)        384       
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 32, 32, 96)        27744     
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 32, 32, 96)        27744     
_________________________________________________________________
batch_normalization_28 (Batc (None, 32, 32, 96)        384       
__________

 753/1562 [=============>................] - ETA: 58s - loss: 4.2273 - categorical_accuracy: 0.0766 - top_k_categorical_accuracy: 0.2366

1487/1562 [===========================>..] - ETA: 5s - loss: 4.1671 - categorical_accuracy: 0.0849 - top_k_categorical_accuracy: 0.2588

1562/1562 [==============================] - 111s 71ms/step - loss: 4.1641 - categorical_accuracy: 0.0852 - top_k_categorical_accuracy: 0.2601
Epoch 2/5
 308/1562 [====>.........................] - ETA: 1:27 - loss: 4.0709 - categorical_accuracy: 0.1005 - top_k_categorical_accuracy: 0.2982

1039/1562 [==================>...........] - ETA: 36s - loss: 4.0453 - categorical_accuracy: 0.1016 - top_k_categorical_accuracy: 0.3008

...

 729/1562 [=============>................] - ETA: 59s - loss: 3.9795 - categorical_accuracy: 0.1168 - top_k_categorical_accuracy: 0.3194

1463/1562 [===========================>..] - ETA: 6s - loss: 3.9651 - categorical_accuracy: 0.1193 - top_k_categorical_accuracy: 0.3270

1562/1562 [==============================] - 110s 71ms/step - loss: 3.9655 - categorical_accuracy: 0.1195 - top_k_categorical_accuracy: 0.3267
Epoch 5/5
 297/1562 [====>.........................] - ETA: 1:28 - loss: 3.9503 - categorical_accuracy: 0.1185 - top_k_categorical_accuracy: 0.3282

1028/1562 [==================>...........] - ETA: 37s - loss: 3.9489 - categorical_accuracy: 0.1191 - top_k_categorical_accuracy: 0.3277

1562/1562 [==============================] - 110s 70ms/step - loss: 3.9442 - categorical_accuracy: 0.1200 - top_k_categorical_accuracy: 0.3298
Accuracy: 0.1273036858974359, Top5 Accuracy 0.33543669871794873 and Loss 3.939175302401567.


## Dephwise Separable Convolutions

In [0]:
# Push that button!
main("sep", verbose=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
separable_conv2d_19 (Separab (None, 32, 32, 96)        411       
_________________________________________________________________
batch_normalization_42 (Batc (None, 32, 32, 96)        384       
_________________________________________________________________
separable_conv2d_20 (Separab (None, 32, 32, 96)        10176     
_________________________________________________________________
batch_normalization_43 (Batc (None, 32, 32, 96)        384       
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 16, 16, 96)        83040     
_________________________________________________________________
batch_normalization_44 (Batc (None, 16, 16, 96)        384       
__________

1363/1562 [=========================>....] - ETA: 9s - loss: 4.1763 - categorical_accuracy: 0.0837 - top_k_categorical_accuracy: 0.2572

1562/1562 [==============================] - 73s 47ms/step - loss: 4.1638 - categorical_accuracy: 0.0852 - top_k_categorical_accuracy: 0.2611
Epoch 2/5
 546/1562 [=========>....................] - ETA: 46s - loss: 4.0528 - categorical_accuracy: 0.1035 - top_k_categorical_accuracy: 0.3019

1562/1562 [==============================] - 71s 46ms/step - loss: 4.0402 - categorical_accuracy: 0.1054 - top_k_categorical_accuracy: 0.3054
Epoch 3/5
 153/1562 [=>............................] - ETA: 1:03 - loss: 3.9979 - categorical_accuracy: 0.1148 - top_k_categorical_accuracy: 0.3172

1525/1562 [============================>.] - ETA: 1s - loss: 3.9947 - categorical_accuracy: 0.1145 - top_k_categorical_accuracy: 0.3175

1562/1562 [==============================] - 71s 45ms/step - loss: 3.9945 - categorical_accuracy: 0.1147 - top_k_categorical_accuracy: 0.3176
Epoch 4/5
 626/1562 [===========>..................] - ETA: 42s - loss: 3.9712 - categorical_accuracy: 0.1210 - top_k_categorical_accuracy: 0.3266

1562/1562 [==============================] - 71s 45ms/step - loss: 3.9656 - categorical_accuracy: 0.1205 - top_k_categorical_accuracy: 0.3289
Epoch 5/5
 204/1562 [==>...........................] - ETA: 1:01 - loss: 3.9374 - categorical_accuracy: 0.1244 - top_k_categorical_accuracy: 0.3441

1562/1562 [==============================] - 71s 45ms/step - loss: 3.9440 - categorical_accuracy: 0.1260 - top_k_categorical_accuracy: 0.3365
Accuracy: 0.13481570512820512, Top5 Accuracy 0.33764022435897434 and Loss 3.9103850561838884.


## Wide Standard Convolutional Neural Network

In [0]:
main("std", net_type="wide", verbose=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_79 (Conv2D)           (None, 32, 32, 192)       5376      
_________________________________________________________________
batch_normalization_66 (Batc (None, 32, 32, 192)       768       
_________________________________________________________________
conv2d_80 (Conv2D)           (None, 16, 16, 96)        165984    
_________________________________________________________________
batch_normalization_67 (Batc (None, 16, 16, 96)        384       
_________________________________________________________________
conv2d_81 (Conv2D)           (None, 16, 16, 384)       332160    
_________________________________________________________________
batch_normalization_68 (Batc (None, 16, 16, 384)       1536      
__________

1434/1562 [==========================>...] - ETA: 6s - loss: 4.2004 - categorical_accuracy: 0.0800 - top_k_categorical_accuracy: 0.2447

1562/1562 [==============================] - 74s 47ms/step - loss: 4.1937 - categorical_accuracy: 0.0805 - top_k_categorical_accuracy: 0.2470
Epoch 2/5
 588/1562 [==========>...................] - ETA: 44s - loss: 4.0982 - categorical_accuracy: 0.0950 - top_k_categorical_accuracy: 0.2790

1562/1562 [==============================] - 72s 46ms/step - loss: 4.0822 - categorical_accuracy: 0.0969 - top_k_categorical_accuracy: 0.2869
Epoch 3/5
 194/1562 [==>...........................] - ETA: 1:03 - loss: 4.0470 - categorical_accuracy: 0.1029 - top_k_categorical_accuracy: 0.2954

1560/1562 [============================>.] - ETA: 0s - loss: 4.0390 - categorical_accuracy: 0.1058 - top_k_categorical_accuracy: 0.3018

1562/1562 [==============================] - 72s 46ms/step - loss: 4.0391 - categorical_accuracy: 0.1058 - top_k_categorical_accuracy: 0.3018
Epoch 4/5
 436/1562 [=======>......................] - ETA: 52s - loss: 4.0135 - categorical_accuracy: 0.1077 - top_k_categorical_accuracy: 0.3117

1562/1562 [==============================] - 72s 46ms/step - loss: 4.0135 - categorical_accuracy: 0.1078 - top_k_categorical_accuracy: 0.3104
Epoch 5/5
 108/1562 [=>............................] - ETA: 1:06 - loss: 3.9906 - categorical_accuracy: 0.1143 - top_k_categorical_accuracy: 0.3235

1489/1562 [===========================>..] - ETA: 3s - loss: 3.9963 - categorical_accuracy: 0.1119 - top_k_categorical_accuracy: 0.3171

1562/1562 [==============================] - 72s 46ms/step - loss: 3.9963 - categorical_accuracy: 0.1118 - top_k_categorical_accuracy: 0.3175
Accuracy: 0.1233974358974359, Top5 Accuracy 0.32662259615384615 and Loss 3.9777390643572197.


## Wide Separable Convolutional Neural Network

In [0]:
main("sep", net_type="wide", verbose=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
separable_conv2d_35 (Separab (None, 32, 32, 192)       795       
_________________________________________________________________
batch_normalization_81 (Batc (None, 32, 32, 192)       768       
_________________________________________________________________
conv2d_95 (Conv2D)           (None, 16, 16, 96)        165984    
_________________________________________________________________
batch_normalization_82 (Batc (None, 16, 16, 96)        384       
_________________________________________________________________
separable_conv2d_36 (Separab (None, 16, 16, 384)       38112     
_________________________________________________________________
batch_normalization_83 (Batc (None, 16, 16, 384)       1536      
__________

1520/1562 [============================>.] - ETA: 1s - loss: 4.1846 - categorical_accuracy: 0.0814 - top_k_categorical_accuracy: 0.2517

1562/1562 [==============================] - 69s 44ms/step - loss: 4.1823 - categorical_accuracy: 0.0821 - top_k_categorical_accuracy: 0.2525
Epoch 2/5
 647/1562 [===========>..................] - ETA: 39s - loss: 4.0784 - categorical_accuracy: 0.0972 - top_k_categorical_accuracy: 0.2917

1562/1562 [==============================] - 67s 43ms/step - loss: 4.0682 - categorical_accuracy: 0.1004 - top_k_categorical_accuracy: 0.2965
Epoch 3/5
 244/1562 [===>..........................] - ETA: 57s - loss: 4.0455 - categorical_accuracy: 0.1104 - top_k_categorical_accuracy: 0.3030

1562/1562 [==============================] - 67s 43ms/step - loss: 4.0309 - categorical_accuracy: 0.1064 - top_k_categorical_accuracy: 0.3086
Epoch 4/5
  58/1562 [>.............................] - ETA: 1:04 - loss: 4.0265 - categorical_accuracy: 0.1051 - top_k_categorical_accuracy: 0.3098

1503/1562 [===========================>..] - ETA: 2s - loss: 4.0093 - categorical_accuracy: 0.1122 - top_k_categorical_accuracy: 0.3133

1562/1562 [==============================] - 67s 43ms/step - loss: 4.0080 - categorical_accuracy: 0.1119 - top_k_categorical_accuracy: 0.3138
Epoch 5/5
 643/1562 [===========>..................] - ETA: 39s - loss: 3.9882 - categorical_accuracy: 0.1171 - top_k_categorical_accuracy: 0.3243

1562/1562 [==============================] - 67s 43ms/step - loss: 3.9925 - categorical_accuracy: 0.1150 - top_k_categorical_accuracy: 0.3218
Accuracy: 0.1266025641025641, Top5 Accuracy 0.33283253205128205 and Loss 3.96610679764014.


## Wide Simply Factorized Neural Network

In [0]:
main("fac", net_type="wide", verbose=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_118 (Conv2D)          (None, 32, 32, 192)       1920      
_________________________________________________________________
conv2d_119 (Conv2D)          (None, 32, 32, 192)       110784    
_________________________________________________________________
batch_normalization_96 (Batc (None, 32, 32, 192)       768       
_________________________________________________________________
conv2d_120 (Conv2D)          (None, 16, 16, 96)        165984    
_________________________________________________________________
batch_normalization_97 (Batc (None, 16, 16, 96)        384       
_________________________________________________________________
conv2d_122 (Conv2D)          (None, 16, 16, 384)       110976    
__________

 762/1562 [=============>................] - ETA: 1:02 - loss: 4.2492 - categorical_accuracy: 0.0679 - top_k_categorical_accuracy: 0.2252

1496/1562 [===========================>..] - ETA: 5s - loss: 4.1985 - categorical_accuracy: 0.0773 - top_k_categorical_accuracy: 0.2454

1562/1562 [==============================] - 120s 77ms/step - loss: 4.1949 - categorical_accuracy: 0.0783 - top_k_categorical_accuracy: 0.2468
Epoch 2/5
 313/1562 [=====>........................] - ETA: 1:33 - loss: 4.0971 - categorical_accuracy: 0.0962 - top_k_categorical_accuracy: 0.2846

1044/1562 [===================>..........] - ETA: 38s - loss: 4.0855 - categorical_accuracy: 0.0950 - top_k_categorical_accuracy: 0.2872

...

 729/1562 [=============>................] - ETA: 1:02 - loss: 4.0187 - categorical_accuracy: 0.1089 - top_k_categorical_accuracy: 0.3087

1464/1562 [===========================>..] - ETA: 7s - loss: 4.0098 - categorical_accuracy: 0.1092 - top_k_categorical_accuracy: 0.3128

1562/1562 [==============================] - 117s 75ms/step - loss: 4.0102 - categorical_accuracy: 0.1094 - top_k_categorical_accuracy: 0.3122
Epoch 5/5
 297/1562 [====>.........................] - ETA: 1:34 - loss: 3.9895 - categorical_accuracy: 0.1142 - top_k_categorical_accuracy: 0.3209

1027/1562 [==================>...........] - ETA: 40s - loss: 3.9977 - categorical_accuracy: 0.1123 - top_k_categorical_accuracy: 0.3162

1562/1562 [==============================] - 117s 75ms/step - loss: 3.9905 - categorical_accuracy: 0.1143 - top_k_categorical_accuracy: 0.3197
Accuracy: 0.11909054487179487, Top5 Accuracy 0.32401842948717946 and Loss 3.9671124288669.


## Failures aside, now a good architecture

In [0]:
class AdequatelyDesignedAllCNNLike:
    """
    isnpired from https://arxiv.org/abs/1412.6806
    namely All-CNN-C
    """
    def __init__(self):
        pass
        
    def make(self, inp, nb_classes):
        # the chunk below can be represented as a single layer with 5x5 kernels
        conv1 = Conv2D(96, (3, 3), padding="same")(inp)
        bn1   = BatchNormalization()(conv1)
        act1  = Activation("relu")(bn1)
        conv2 = Conv2D(96, (3, 3), padding="same")(act1)
        bn2   = BatchNormalization()(conv2)
        act2  = Activation("relu")(bn2)
        
        conv3 = Conv2D(96, (3, 3), padding="same", strides=(2, 2))(act2)
        bn3   = BatchNormalization()(conv3)
        act3  = Activation("relu")(bn3)
        
        conv4 = SeparableConv2D(256, (3, 3), padding="same")(act3)
        bn4   = BatchNormalization()(conv4)
        act4  = Activation("relu")(bn4)
        
        conv5 = Conv2D(192, (3, 3), padding="same", strides=(2, 2))(act4)
        bn5   = BatchNormalization()(conv5)
        act5  = Activation("relu")(bn5)
        
        conv6 = SeparableConv2D(256, (3, 3), padding="same")(act5)
        bn6   = BatchNormalization()(conv6)
        act6  = Activation("relu")(bn6)
        
        conv7 = Conv2D(nb_classes, (1, 1), padding="same")(act6)
        bn7   = BatchNormalization()(conv7)
        act7  = Activation("relu")(bn7)
        
        gap = GlobalAveragePooling2D()(act7)
        final = Activation("softmax")(gap)
        
        return Model(inputs=inp, outputs=final)

In [0]:
inp = Input(shape=(32, 32, 3))
net = AdequatelyDesignedAllCNNLike().make(inp, 100)
net.compile(SGD(lr=0.01, decay=0.001, momentum=0.8), "categorical_crossentropy",
             metrics=[categorical_accuracy, top_k_categorical_accuracy])


print(net.summary())

net.fit_generator(train_gen, steps_per_epoch=50000 // BATCH_SIZE, epochs=5)
loss, acc, topk_acc = net.evaluate_generator(test_gen, steps=10000 // BATCH_SIZE)

print(f"Accuracy: {acc}, Top5 Accuracy {topk_acc} and Loss {loss}.")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_127 (Conv2D)          (None, 32, 32, 96)        2688      
_________________________________________________________________
batch_normalization_102 (Bat (None, 32, 32, 96)        384       
_________________________________________________________________
activation_115 (Activation)  (None, 32, 32, 96)        0         
_________________________________________________________________
conv2d_128 (Conv2D)          (None, 32, 32, 96)        83040     
_________________________________________________________________
batch_normalization_103 (Bat (None, 32, 32, 96)        384       
_________________________________________________________________
activation_116 (Activation)  (None, 32, 32, 96)        0         
__________

1315/1562 [========================>.....] - ETA: 11s - loss: 4.0162 - categorical_accuracy: 0.1178 - top_k_categorical_accuracy: 0.3317

1562/1562 [==============================] - 75s 48ms/step - loss: 3.9747 - categorical_accuracy: 0.1251 - top_k_categorical_accuracy: 0.3458
Epoch 2/5
 493/1562 [========>.....................] - ETA: 49s - loss: 3.6405 - categorical_accuracy: 0.1930 - top_k_categorical_accuracy: 0.4620

1562/1562 [==============================] - 73s 47ms/step - loss: 3.5535 - categorical_accuracy: 0.2068 - top_k_categorical_accuracy: 0.4875
Epoch 3/5
 102/1562 [>.............................] - ETA: 1:07 - loss: 3.4211 - categorical_accuracy: 0.2396 - top_k_categorical_accuracy: 0.5251

1394/1562 [=========================>....] - ETA: 7s - loss: 3.3254 - categorical_accuracy: 0.2500 - top_k_categorical_accuracy: 0.5525

1562/1562 [==============================] - 73s 47ms/step - loss: 3.3173 - categorical_accuracy: 0.2518 - top_k_categorical_accuracy: 0.5542
Epoch 4/5
 525/1562 [=========>....................] - ETA: 48s - loss: 3.2006 - categorical_accuracy: 0.2679 - top_k_categorical_accuracy: 0.5852

1562/1562 [==============================] - 73s 47ms/step - loss: 3.1602 - categorical_accuracy: 0.2810 - top_k_categorical_accuracy: 0.5942
Epoch 5/5
 112/1562 [=>............................] - ETA: 1:07 - loss: 3.0590 - categorical_accuracy: 0.3061 - top_k_categorical_accuracy: 0.6169

1458/1562 [===========================>..] - ETA: 4s - loss: 3.0364 - categorical_accuracy: 0.3096 - top_k_categorical_accuracy: 0.6198

1562/1562 [==============================] - 72s 46ms/step - loss: 3.0324 - categorical_accuracy: 0.3102 - top_k_categorical_accuracy: 0.6208
Accuracy: 0.1837940705128205, Top5 Accuracy 0.4434094551282051 and Loss 3.6364160730288577.


## Final notes

As you can see, it is indeed possible with slight alterations to speed up a convolutional network like All-CNN-C by 25% (See Wide Separable Convolution) with no significant drop in accuracy.
Moreover, with a better-designed network (See AdequatelyDesignedAllCNNLike model), the speedup is still considerable (> 15%) but the accuracy increases almost 1.5x, after the same amount of time!

Anyway, remember that there's no silver bullet and you should always experiment in order to get satisfying results.

Hopefully, the methods showed above and described in the blog post will help.